In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:11:24 of 0:11:25 remaining]
Chain 1:  10% [0:00:59 of 0:01:06 remaining]
Chain 1:  20% [0:01:02 of 0:01:18 remaining]
Chain 1:  30% [0:00:55 of 0:01:19 remaining]
Chain 1:  40% [0:00:48 of 0:01:20 remaining]
Chain 1:  50% [0:00:41 of 0:01:21 remaining]
Chain 1:  60% [0:00:33 of 0:01:22 remaining]
Chain 1:  70% [0:00:25 of 0:01:24 remaining]
Chain 1:  80% [0:00:17 of 0:01:24 remaining]
Chain 1:  90% [0:00:08 of 0:01:23 remaining]
Chain 1: 100% [0:00:00 of 0:01:22 remaining]

Chain 2:   0% [0:00:03 of 0:00:03 remaining]
Chain 2:  10% [0:00:37 of 0:00:41 remaining]
Chain 2:  20% [0:00:49 of 0:01:01 remaining]
Chain 2:  30% [0:00:45 of 0:01:05 remaining]
Chain 2:  40% [0:00:40 of 0:01:07 remaining]
Chain 2:  50% [0:00:34 of 0:01:08 remaining]
Chain 2:  60% [0:00:27 of 0:01:09 remaining]
Chain 2:  70% [0:00:21 of 0:01:09 remaining]
Chain 2:  80% [0:00:14 of 0:01:09 remaining]
Chain 2:  90% [0:00:07 of 0:01:09 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[25.2848 112.756 0.922786; 25.7527 111.26 0.9587; … ; 28.008 112.87 0.926036; 27.5518 112.576 0.940194]

[25.2583 115.247 0.870461; 23.0771 109.491 1.00263; … ; 26.8415 113.247 0.926967; 26.5063 116.668 0.843669]

[29.739 113.228 0.924762; 22.604 115.192 0.883504; … ; 23.6612 113.955 0.908885; 26.3371 113.954 0.898383]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean          SD        Naive SE       MCSE        ESS   
     s2  26.15802727 1.989259687 0.01210624893 0.0191562542 9000.0000
beta[1] 113.89805476 1.974686810 0.01201756122 0.0370529913 2840.2064
beta[2]   0.90465857 0.043477684 0.00026459676 0.0008074527 2899.3365

Quantiles:
            2.5%        25.0%        50.0%       75.0%       97.5%   
     s2  22.5281217  24.77764752  26.07315568  27.4422523  30.3044994
beta[1] 110.0554543 112.55972965 113.90215901 115.2449892 117.7815786
beta[2]   0.8190722   0.87522166   0.90451113   0.9341674   0.9900263



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*